In [1]:
# conectarse a Azure storage datalake
! pip install azure-storage-file-datalake

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 3.5/3.5 MB 27.8 MB/s eta 0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl (107 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from io import StringIO

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

# Nombre global del Storage Account
storage_account = os.getenv("AZURE_STORAGE_ACCOUNT")
# "llave" temporal. Se genera en el portal de Azure bajo "Shared access signature".
sas_token = os.getenv("AZURE_SAS_TOKEN")
# carpeta raíz o "cubeta" donde residen tus datos.
container = os.getenv("AZURE_CONTAINER")
# Nombre exacto del archivo, incluyendo la ruta si está en subcarpeta
file_name = os.getenv("AZURE_FILE_NAME")

In [ ]:
def obtener_cliente_adls():
    client = DataLakeServiceClient(
        account_url= f"https://{storage_account}.dfs.core.windows.net",
        credential=sas_token
    )
    return client

In [ ]:
client = obtener_cliente_adls()
container_client = client.get_file_system_client(file_system=container)
file = container_client.get_file_client(file_name)

In [ ]:
download = file.download_file()
download_bytes = download.readall()
file_content = download_bytes.decode("utf-8")
file_content

In [ ]:
df = pd.read_csv(StringIO(file_content))

Otra versión del Código:

In [ ]:
import os
from io import BytesIO
from typing import Optional

import pandas as pd
from dotenv import load_dotenv
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core.exceptions import AzureError

In [ ]:
# Cargamos las variables al inicio del script
load_dotenv()

In [ ]:
def get_adls_client() -> DataLakeServiceClient:
    """
    Configura el cliente de Azure Data Lake.
    Las variables se buscan primero en el entorno del sistema y luego en el .env
    """
    account_name: Optional[str] = os.getenv("AZURE_STORAGE_ACCOUNT")
    sas_token: Optional[str] = os.getenv("AZURE_STORAGE_SAS_TOKEN")
    
    # Validación Senior: Falla rápido y con mensajes claros
    if not account_name or not sas_token:
        raise EnvironmentError(
            "ERROR: No se encontraron las credenciales. "
            "Asegúrate de tener un archivo .env configurado o las variables de entorno definidas."
        )

    account_url = f"https://{account_name}.dfs.core.windows.net"
    
    return DataLakeServiceClient(account_url=account_url, credential=sas_token)

def download_and_read_csv(container_name: str, file_path: str) -> pd.DataFrame:
    """Descarga un archivo desde ADLS Gen2 y lo carga en un DataFrame."""
    try:
        service_client = get_adls_client()
        file_system_client = service_client.get_file_system_client(file_system=container_name)
        file_client = file_system_client.get_file_client(file_path)

        # No leemos todo en una variable. 
        # Usamos un stream de bytes directo a Pandas.
        # Descarga eficiente mediante streaming de bytes
        download = file_client.download_file()
        
        # Leemos los bytes y los pasamos directamente a un buffer
        # Esto es mucho más eficiente que convertir a string manualmente
        stream = BytesIO(download.readall())
        
        return pd.read_csv(stream)

    except AzureError as e:
        # Aquí podrías implementar un sistema de logging real (logging.error)
        print(f"Error de infraestructura Azure: {e}")
        raise
    except Exception as e:
        print(f"Error inesperado al procesar el CSV: {e}")
        raise

if __name__ == "__main__":
    # Estos valores podrían venir también de variables de entorno si cambian seguido
    CONTAINER = "raw-data"
    FILE_NAME = "ventas_2024.csv"
    
    try:
        df = download_and_read_csv(CONTAINER, FILE_NAME)
        print("Carga exitosa. Vista previa:")
        print(df.head())
    except Exception as e:
        print(f"El proceso falló: {e}")